# 12.2 고급 GroupBy 사용

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

In [3]:
# 10장에서 Series와 DataFrame에 대해 gorupby 메서드를 사용하는 방법을 이미 살펴봤지만 유용한 몇 가지 기법을 더 소개하겠다.

- 12.2.1 그룹 변환과 GroupBy 객체 풀어내기

In [4]:
# 10장에서는 그룹 연산에 apply 메서드를 이용해서 DataFrame을 변환하는 방법을 살펴봤다.
# transform이라는 내장 메서드를 이용하면 apply 메서드와 유사하게 동작하면서도 사용할 수 있는 함수의 종류에 대해 좀 더 많은 제한을 포함할 수 있다.

- 그룹 형태로 브로드캐스트할 수 있는 스칼라값을 생성해야 한다.
- 입력 그룹과 같은 형태의 객체를 반환해야 한다.
- 입력을 변경하지 않아야 한다.

In [5]:
# 설명을 위해 간단한 예제를 살펴보자.

In [6]:
df = pd.DataFrame({"key": ["a", "b", "c"] * 4,
                   "value": np.arange(12.)})

In [7]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [8]:
# key에 따른 그룹의 평균을 구해보자.

In [9]:
g = df.groupby('key').value

In [10]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [11]:
# df["value"]와 같은 형태의 Series를 원한 것이 아니라 "key"에 따른 그룹의 평균값으로 값을 변경하기 원했다고 가정한다면,
# transform에 람다 함수 lambda x: x.mean()을 넘기면 된다.

In [12]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [13]:
# 내장 요약함수에 대해서는 agg 메서드에서처럼 문자열 그룹 연산 이름을 넘기면 된다.

In [14]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [15]:
# apply와 마찬가지로 transform은 Series를 반환하는 함수만 사용할 수 있지만 결과는 입력과 똑같은 크기여야 한다.
# 예를 들어 람다 함수를 이용해서 각 그룹에 모두 2를 곱할 수 있다.

In [16]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [17]:
# 좀 더 복잡한 예제로, 각 그룹에 대해 내림차순으로 순위를 계산할 수도 있다.

In [18]:
g.transform(lambda x: x.rank(ascending=True))

0     1.0
1     1.0
2     1.0
3     2.0
4     2.0
5     2.0
6     3.0
7     3.0
8     3.0
9     4.0
10    4.0
11    4.0
Name: value, dtype: float64

In [19]:
# 간단한 요약을 통해 그룹 변환을 수행하는 함수를 살펴보자.

In [20]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [21]:
# 이 경우에는 transform이나 apply를 이용해서 같은 결과를 얻을 수 있다.

In [22]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [23]:
g.apply(normalize)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_46828\400169470.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  g.apply(normalize)


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [24]:
# mean이나 sum 같은 내장 요약함수는 일반적인 apply 함수보다 더 빠르게 동작한다.
# 또한 이 함수들을 transform과 함께 사용하면 뒤로 되돌릴 수 있는데 이를 통해 그룹 연산을 풀어낼 수 있다.

In [25]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [26]:
normalized = (df["value"] - g.transform("mean")) / g.transform("std")

In [27]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [28]:
# 그룹 연산을 풀어내면 수차례의 그룹 연산을 수행하게 되지만 전체 벡터 연산의 장점이 더 크다.

- 12.2.2 시계열 그룹 리샘플링

In [29]:
# 시계열 데이터에서 resample 메서드는 의미적으로 시간 간격에 기반한 그룹 연산이다.
# 다음 예제를 살펴보자.

In [30]:
N = 15

In [31]:
times = pd.date_range("2017-05-20 00:00", freq="1min", periods=N)

In [32]:
df = pd.DataFrame({"time": times,
                   "value": np.arange(N)})

In [33]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [34]:
# 여기서 time으로 색인한 후 리샘플해보자.

In [35]:
df.set_index("time").resample("5min").count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [36]:
# key 컬럼으로 구분되는 여러 시계열 데이터를 담고 있는 DataFrame을 생각해보자.

In [37]:
df2 = pd.DataFrame({"time": times.repeat(3),
                    "key": np.tile(["a", "b", "c"], N), 
                    "value": np.arange(N * 3.)})

In [38]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [39]:
# "key"의 각 값에 대해 같은 리샘플을 수행하기 위해서는 pandas.TimeGrouper 객체를 이용한다. - pandas 버전 업데이트로 인해 사용불가 
# 이후 코드 작성에 어려움이 있음. 해결 예정

In [48]:
time_key = pd.Grouper(level="time", freq="5T") # level 파라미터를 사용, key 파라미터는 str 자료형에 이용

In [49]:
# 그리고 time을 색인으로 한 다음 "key"와 time_key로 그룹지어 합을 구해보자.

In [50]:
resampled = (df2.set_index("time")        # 오류 발생, time_key에서 level 파라미터 사용 후 해결
              .groupby(["key", time_key])
              .sum())

In [51]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [52]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


In [53]:
# Grouper를 사용할 때 주의해야 할 점은 시간값이 Series 혹은 DataFrame의 색인이어야 한다는 점이다.